In [10]:
import os
import pandas as pd
import numpy as np
import quandl
import csv

In [11]:
data = pd.DataFrame()
data = quandl.get("WIKI/AAPL", start_date="1950-01-01", end_date="2018-02-23", api_key='zqxuPnVG5igk65xLKwKQ')

data

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
1980-12-12,28.75,28.87,28.75,28.750,2093900.0,0.0,1.0,0.422706,0.424470,0.422706,0.422706,117258400.0
1980-12-15,27.38,27.38,27.25,27.250,785200.0,0.0,1.0,0.402563,0.402563,0.400652,0.400652,43971200.0
1980-12-16,25.37,25.37,25.25,25.250,472000.0,0.0,1.0,0.373010,0.373010,0.371246,0.371246,26432000.0
1980-12-17,25.87,26.00,25.87,25.870,385900.0,0.0,1.0,0.380362,0.382273,0.380362,0.380362,21610400.0
1980-12-18,26.63,26.75,26.63,26.630,327900.0,0.0,1.0,0.391536,0.393300,0.391536,0.391536,18362400.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2018-02-16,172.36,174.82,171.77,172.430,39638793.0,0.0,1.0,172.360000,174.820000,171.770000,172.430000,39638793.0
2018-02-20,172.05,174.26,171.42,171.850,33531012.0,0.0,1.0,172.050000,174.260000,171.420000,171.850000,33531012.0
2018-02-21,172.83,174.12,171.01,171.070,35833514.0,0.0,1.0,172.830000,174.120000,171.010000,171.070000,35833514.0


In [12]:
path = str(os.getcwd()) + "/" + "data"
data.to_csv("trainData.csv")

In [13]:
def read_train():
    prices = pd.read_csv("trainData.csv")
    prices = clean(prices)
    return prices

def clean(prices):
    (m, n) = prices.shape

    prices = prices.loc[:, ['Date', 'Open', 'Close']]

    # Add new column "Close Price at the Previous Day" and "Open Price at the Previous Day"
    prevColumn = prices.loc[:, ['Open', 'Close']].assign(index = lambda x: x.index-1)
    prevColumn = prevColumn.set_index('index')
    prices = prices.join(prevColumn, rsuffix='Prev')

    # Add new column "Close Price at the Day Before" and "Open Price at the Day Before"
    prevColumn = prices.loc[:, ['Open', 'Close']].assign(index = lambda x: x.index-2)
    prevColumn = prevColumn.set_index('index')
    prices = prices.join(prevColumn, rsuffix='Prev2')

    prevColumn = prices.loc[:, ['Open', 'Close']].assign(index = lambda x: x.index-3)
    prevColumn = prevColumn.set_index('index')
    prices = prices.join(prevColumn, rsuffix='Prev3')

    prevColumn = prices.loc[:, ['Open', 'Close']].assign(index = lambda x: x.index-4)
    prevColumn = prevColumn.set_index('index')
    prices = prices.join(prevColumn, rsuffix='Prev4')

    prevColumn = prices.loc[:, ['Open', 'Close']].assign(index = lambda x: x.index-5)
    prevColumn = prevColumn.set_index('index')
    prices = prices.join(prevColumn, rsuffix='Prev5')



    # No need now to save the fifth days data
    prices = prices[:m-5]

    return prices


def write_train(data):
    data.loc[:, ['Date','Open','OpenPrev','ClosePrev','OpenPrev2','ClosePrev2','OpenPrev3','ClosePrev3',
                 'OpenPrev4','ClosePrev4','OpenPrev5','ClosePrev5']].to_csv("trainX.csv", index=False)
    data.loc[:, ['Close']].to_csv( "trainY.csv",index=False)
    
def write_test(data):
    print(year_Y_2017)
    year_2017=data.loc[mask]
    year_2017.loc[:, ['Date','Open','OpenPrev','ClosePrev','OpenPrev2','ClosePrev2','OpenPrev3','ClosePrev3',
                 'OpenPrev4','ClosePrev4','OpenPrev5','ClosePrev5']].to_csv("trainX.csv", index=False)
    year_2017.loc[:, ['Close']].to_csv( "trainY.csv",index=False)

In [14]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import explained_variance_score

In [15]:
def read_val():
    X = pd.read_csv("trainX.csv", header=0)
    y = pd.read_csv("trainY.csv", header=0)
    X['Date'] = pd.to_datetime(X['Date'], format='%Y-%m-%d')
    return X, y



def plot_trend(x, predict, y, title):
    plt.xticks(rotation=45)
    plt.plot_date(x, y, fmt='b-', xdate=True, ydate=False, label='Real value')
    plt.plot_date(x, predict, fmt='r-', xdate=True, ydate=False, label='Predicted value')
    plt.legend(loc='upper center')
    plt.ylabel('Close prices')
    plt.title(title)
    plt.grid()
    plt.show()


def modify(X, columns):
    columns.append('Date2num')
    returnX = X.loc[:, columns]
    return returnX


def predict_and_plot(X, date):
    m, n = X.shape
    X_test = X.loc[np.floor(m*0.6)+1:]
    X_train = X.loc[:np.floor(m*0.6)]
    
    y_test = y.loc[np.floor(m*0.6)+1:]
    y_train = y.loc[:np.floor(m*0.6)]
    
    date_test = date.loc[np.floor(m*0.6)+1:]
    date_train = date.loc[:np.floor(m*0.6)]
    
    lr = LinearRegression()

    lr.fit(X_train, y_train)

    print('Coefficients: \n', lr.coef_)
    print('Intercept: \n', lr.intercept_)
    pred = lr.predict(X_test)
#     mae = mean_absolute_error(y_test,pred)
    
#     min_data = pred-mae
#     max_data = pred+mae
#     #my_data.append(pred-mae)
#     #print(my_data)
#     data =[{'Min' : min_data, 'Max' : max_data}]
#     min_max_range = pd.DataFrame(data)
#     #min_max_range['Max'] = pred+mae
#     print(min_max_range)
    
    
#     print("Number of variables: %d; Residual sum of squares: %.2f; Variance score: %.5f,Mean absolute error: %.2f"
#           % (n, np.mean((pred - y_test) ** 2), explained_variance_score(y_test, pred),mean_absolute_error(y_test,pred)))
   
    
#     plot_trend(date_train, lr.predict(X_train), y_train, 'Apple')
#     plot_trend(date_test, lr.predict(X_test), y_test, 'Apple')
    
    return lr,pred


In [16]:
prices_train=read_train()
write_train(prices_train)

X, y = read_val()

X['Date2num'] = X['Date'].apply(lambda x: mdates.date2num(x))
date = X.loc[:, ['Date']]
del X['Date']

X1 = modify(X, [])
predict_and_plot(X1,date)

Coefficients: 
 [[0.00076645]]
Intercept: 
 [-521.22946147]


(LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False),
 array([[39.27815053],
        [39.27891698],
        [39.27968344],
        ...,
        [43.44612591],
        [43.44689236],
        [43.44765881]]))

In [17]:
#X2 = modify(X, ['Open'])
#lr,close_val=predict_and_plot(X2, date)

X3 = modify(X, ['Open','OpenPrev','ClosePrev'])
lr,close_val=predict_and_plot(X3, date)
X3

Coefficients: 
 [[4.53114954e-01 5.35639322e-01 1.29750833e-02 3.52903695e-06]]
Intercept: 
 [-2.63869268]


,Open,OpenPrev,ClosePrev,Date2num
0,28.750,27.380,27.25,723161.0
1,27.380,25.370,25.25,723164.0
2,25.370,25.870,25.87,723165.0
3,25.870,26.630,26.63,723166.0
4,26.630,28.250,28.25,723167.0
...,...,...,...,...
9368,157.070,158.500,162.71,736734.0
9369,158.500,161.950,164.34,736737.0
9370,161.950,163.045,167.37,736738.0
9371,163.045,169.790,172.99,736739.0


In [18]:
daterange = pd.date_range('2018-02-26', '2018-04-30', freq='B')
newDate = pd.DataFrame(data=daterange, columns=['Date'])
newDate = newDate.dropna()
newDate

,Date
0,2018-02-26
1,2018-02-27
2,2018-02-28
3,2018-03-01
4,2018-03-02
5,2018-03-05
6,2018-03-06
7,2018-03-07
8,2018-03-08
9,2018-03-09


In [19]:
prevOpen = data.loc[:,['Open']]
prevOpen = prevOpen.iloc[len(prevOpen)-1]
prevClose = close_val.item(len(close_val)-1)
prevOpen = prevOpen.values.item(0)
newX3 = X3
try:
    newX3.drop('Date2num',axis=1,inplace=True)
except:
    print('already dropped')
dummy = pd.DataFrame([[prevClose,prevOpen,prevClose]], columns=['Open','OpenPrev','ClosePrev'])
newX3 = newX3.append(dummy, ignore_index=True)
date = date.append(newDate, ignore_index=True)
for x in range(0,len(newDate)-1):
    print(len(newX3))
    print(len(date))
    lr,close_val = predict_and_plot(newX3, date)
    prevClose = close_val.item(len(close_val)-1)
    prevOpen = newX3.loc[:,['Open']]
    prevOpen = prevOpen.iloc[len(prevOpen)-1]
    prevOpen = prevOpen.values.item(0)
    dummy = pd.DataFrame([[prevClose,prevOpen,prevClose]], columns=['Open','OpenPrev','ClosePrev'])
    newX3 = newX3.append(dummy, ignore_index=True)

newX3

9374
9419
Coefficients: 
 [[0.45316599 0.53559317 0.01301275]]
Intercept: 
 [-0.07381775]
9375
9419
Coefficients: 
 [[0.45316392 0.53558952 0.01301653]]
Intercept: 
 [-0.07371714]
9376
9419
Coefficients: 
 [[0.45316392 0.53558952 0.01301653]]
Intercept: 
 [-0.07371714]
9377
9419
Coefficients: 
 [[0.45316359 0.53559292 0.01301061]]
Intercept: 
 [-0.07356921]
9378
9419
Coefficients: 
 [[0.45316359 0.53559292 0.01301061]]
Intercept: 
 [-0.07356921]
9379
9419
Coefficients: 
 [[0.4531633  0.53559375 0.01301091]]
Intercept: 
 [-0.07361274]
9380
9419
Coefficients: 
 [[0.45316477 0.53559574 0.01300839]]
Intercept: 
 [-0.07366091]
9381
9419
Coefficients: 
 [[0.45316477 0.53559574 0.01300839]]
Intercept: 
 [-0.07366091]
9382
9419
Coefficients: 
 [[0.45316122 0.53559896 0.01300664]]
Intercept: 
 [-0.07355334]
9383
9419
Coefficients: 
 [[0.45316122 0.53559896 0.01300664]]
Intercept: 
 [-0.07355334]
9384
9419
Coefficients: 
 [[0.45316131 0.53559884 0.01300694]]
Intercept: 
 [-0.07356739]
9385
9419


,Open,OpenPrev,ClosePrev
0,28.750000,27.380000,27.250000
1,27.380000,25.370000,25.250000
2,25.370000,25.870000,25.870000
3,25.870000,26.630000,26.630000
4,26.630000,28.250000,28.250000
...,...,...,...
9414,178.599264,178.442181,178.599264
9415,178.756635,178.599264,178.756635
9416,178.914129,178.756635,178.914129
9417,179.072039,178.914129,179.072039


In [20]:
date


,Date
0,1980-12-12
1,1980-12-15
2,1980-12-16
3,1980-12-17
4,1980-12-18
...,...
9414,2018-04-24
9415,2018-04-25
9416,2018-04-26
9417,2018-04-27
